<a href="https://colab.research.google.com/github/JoeOlang/NLP/blob/main/Text%20Classification/To%20Vaccinate/v3_lr_and_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import pandas as pd
import numpy as np
import nltk
import spacy
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [7]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Zindi/To Vaccinate/Train.csv").dropna()
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Zindi/To Vaccinate/Test.csv").dropna()


# Creating the input for the pipeline
X_train = train['safe_text'].tolist()
Y_train = train['label'].tolist()

X_test = test['safe_text'].tolist()


In [22]:
def asignval(x):
    if x == 1:
        y = 'Positive'
    elif x == 0:
        y = 'Neutral'
    else:
        y = 'Negative'

    return y


In [28]:
del train['tweet_id']
del train['agreement']
del train['label']

In [30]:
train.groupby(['target'], as_index=False).count()

,target,safe_text
0,Negative,1038
1,Neutral,4908
2,Positive,4053


In [23]:
train['target'] = train['label'].apply(lambda x: asignval(x))

In [34]:
train

,safe_text,target,clean_text
0,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,Neutral,amp big homie meanboy3000 #meanboy #mb #mbs #m...
1,I'm 100% thinking of devoting my career to pro...,Positive,100 thinking devoting career proving autism ca...
2,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",Negative,#whatcausesautism vaccines vaccinate child
3,I mean if they immunize my kid with something ...,Negative,mean immunize kid something secretly kill year...
4,Thanks to <user> Catch me performing at La Nui...,Neutral,thanks user catch performing la nuit nyc 1134 ...
...,...,...,...
9996,Living in a time where the sperm I used to was...,Positive,living time sperm used waste jenny mccarthy be...
9997,<user> <user> In spite of all measles outbrea...,Positive,user user spite measles outbreaks judge mi thr...
9998,Interesting trends in child immunization in Ok...,Neutral,interesting trends child immunization oklahoma...
9999,CDC Says Measles Are At Highest Levels In Deca...,Neutral,cdc says measles highest levels decades url re...


In [32]:
s_character = re.compile('[/(){}\[\]\|@,;]')
symbols = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [33]:
def clean_text(text):
    text = text.lower()
    text = s_character.sub(' ', text)
    text = symbols.sub(' ', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

train['clean_text'] = train['safe_text'].apply(clean_text)

In [38]:
X = train.clean_text
y = train.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [39]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6999,), (3000,), (6999,), (3000,))



---



In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

lr = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression()),
               ])

lr.fit(X_train, y_train)
y_pred1 = lr.predict(X_test)

print(f"Accuracy is : {accuracy_score(y_pred1, y_test)}")

Accuracy is : 0.728




---



In [43]:
from xgboost import XGBClassifier

xgboost = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', LogisticRegression()),
                    ])

xgboost.fit(X_train, y_train)

y_pred = xgboost.predict(X_test)
print(f"Accuracy is : {accuracy_score(y_pred, y_test)}")

Accuracy is : 0.728




---



In [45]:
vect = CountVectorizer()
trans = TfidfTransformer()

In [54]:
len(y_pred)

3000

In [56]:
to_predict = ["Predicted probabilities of demo input string are"]


print('Predicted probabilities of demo input string are')
print(xgboost.predict(to_predict))

Predicted probabilities of demo input string are
['Neutral']


In [59]:
test['pred'] = test['safe_text'].apply(lambda x: xgboost.predict([x]))

In [62]:
def asignval2(x):
    if x == ['Positive']:
        y = 1
    elif x == ['Neutral']:
        y = 0
    else:
        y = -1

    return y

In [63]:
test['label'] = test['pred'].apply(lambda x: asignval2([x]))

In [65]:
del test['pred']
del test['safe_text']

In [66]:
test

,tweet_id,label
0,00BHHHP1,1
1,00UNMD0E,1
2,01AXPTJF,0
3,01HOEQJW,1
4,01JUKMAO,1
...,...,...
5172,ZXVVNC5O,1
5173,ZYIANVI8,0
5174,ZYITEHAH,0
5175,ZZ3BMBTG,1


In [67]:
test.to_csv(r'/content/drive/MyDrive/Colab Notebooks/Zindi/To Vaccinate/Submission1.csv', index = False)